In [12]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
pixel_values = processor(image, return_tensors="pt").pixel_values
print(pixel_values.shape)

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

c:\Users\infin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\infin\.cache\huggingface\hub\models--microsoft--trocr-base-printed. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

torch.Size([1, 3, 384, 384])


In [14]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
import os
import pandas as pd
from PIL import Image
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define paths
data_csv_path = '../../data/data-cross-section.csv'
images_dir = '../../data/preprocessed'
output_csv_path = '../../data/data-extracted.csv'

# Load CSV file
df = pd.read_csv(data_csv_path)

def extract_text_from_image(image_path):
    # Load image
    image = Image.open(image_path).convert("RGB")
    
    # Preprocess image
    pixel_values = processor(image, return_tensors="pt").pixel_values
    
    # Perform OCR
    with torch.no_grad():
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return generated_text

# Ensure 'extracted_text' column exists in DataFrame
if 'extracted_text' not in df.columns:
    df['extracted_text'] = ''

# Iterate through folders and images
for index, row in df.iterrows():
    folder_path = os.path.join(images_dir, str(index))
    
    if os.path.exists(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            extracted_text = extract_text_from_image(image_path)
            
            # Append extracted text to the existing text in the DataFrame
            current_text = df.at[index, 'extracted_text']
            if current_text:
                df.at[index, 'extracted_text'] = f"{current_text} {extracted_text}"
            else:
                df.at[index, 'extracted_text'] = extracted_text

# Save the updated DataFrame to a new CSV file
df.to_csv(output_csv_path, index=False)

print(f"Updated data saved to {output_csv_path}")

df

c:\Users\infin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1128: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Updated data saved to ../../data/data-extracted.csv


,index,image_link,group_id,entity_name,extracted_text
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,2.63IN 6.68CM 36N - 75N 91.44CM - 199.39CM
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"SIZE WIDTH LENGTH 42CM/16.54"" 200CM/78.74"" ONE..."
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,"SIZE WIDTH LENGTH 42CM/16.54"" 200CM/78.74"" ONE..."
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,"SIZE WIDTH LENGTH 42CM/16.54"" 200CM/78.74"" ONE..."
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,"SIZE WIDTH LENGTH 90CM/35.43"" ONE SIZE 10.50CM..."
5,5,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,height,"SIZE WIDTH LENGTH 90CM/35.43"" ONE SIZE 10.50CM..."
6,6,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,width,"SIZE WIDTH LENGTH 90CM/35.43"" ONE SIZE 10.50CM..."
7,7,https://m.media-amazon.com/images/I/11lshEUmCr...,156839,height,3.56CM 5.49 137.16CM
8,8,https://m.media-amazon.com/images/I/21+i52HRW4...,478357,width,30 CM 40 CM 15 CM
9,9,https://m.media-amazon.com/images/I/21-LmSmehZ...,478357,height,30 CM 40 CM 15 CM
